In [1]:
# ──────────────────────────────────────────────────────────────
#  Standard library                                             │
# ──────────────────────────────────────────────────────────────
from typing import List, Optional

from pydantic import Field

# ──────────────────────────────────────────────────────────────
#  SymbolicAI core                                              │
# ──────────────────────────────────────────────────────────────
from symai import Expression           # Base class for your LLM “operators”
from symai.models import LLMDataModel  # Thin Pydantic wrapper w/ LLM hints
from symai.strategy import contract    # The Design-by-Contract decorator

/Users/futurisold/miniconda3/envs/symai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
#  1.  Data models                                             ▬
#     – clear structure + rich Field descriptions power        ▬
#       validation, automatic prompt templating & remedies     ▬
# ▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬

class Document(LLMDataModel):
    """Represents an entire document in the tiny in-memory corpus."""
    id: str = Field(description="Unique identifier of the document.")
    content: str = Field(description="The full raw text of the document.")


class DocSnippet(LLMDataModel):
    """
    Exact passage taken *verbatim* from a Document.
    We store the `doc_id` so the answer can cite its source.
    """
    doc_id: str = Field(description="ID of the document the snippet comes from.")
    snippet: str = Field(description="A short excerpt supporting the answer.")


class MultiDocQAInput(LLMDataModel):
    """
    The *input* to the contract call:
      • the user’s natural-language question
      • the corpus it may answer from
      • a caller-specified upper bound on how many snippets can be cited
    """
    query: str = Field(description="User question in plain English.")
    documents: List[Document] = Field(description="Corpus to search for answers.")
    max_snippets: Optional[int] = Field(
        default=3,
        ge=1,
        le=10,
        description="Max number of snippets the agent may cite (defaults to 3).",
    )


class IntermediateRetrieved(LLMDataModel):
    """
    Returned by `act()`: lightweight retrieval result that will be fed to
    the LLM so it can see relevant sentences without scanning whole docs.
    """
    query: str = Field(description="The original question from the user.")
    top_docs: List[Document] = Field(description="Top-k most relevant documents.")
    selected_sentences: List[str] = Field(
        description="Sentences deemed most relevant to the query."
    )
    target_snippet_count: int = Field(
        description="Upper bound on evidence snippets (copied from input)."
    )


class AnswerWithEvidence(LLMDataModel):
    """
    Final object returned to the **caller** (and validated by `post`).
    """
    answer: str = Field(description="Concise, stand-alone answer.")
    evidence: List[DocSnippet] = Field(description="Cited supporting passages.")
    coverage_score: float = Field(
        ge=0.0,
        le=1.0,
        description=(
            "LLM-estimated fraction of answer that is directly supported by the "
            "evidence (0 = no support, 1 = fully supported)."
        ),
    )

# ▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
#  2.  The contracted class                                    ▬
# ▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
@contract(
    # ── Remedies ─────────────────────────────────────────── #
    pre_remedy=True,      # Try to fix bad inputs automatically
    post_remedy=True,     # Try to fix bad LLM outputs automatically
    accumulate_errors=True,  # Feed history of errors to each retry
    verbose=True,         # Log internal steps (see `symai.strategy` logger)
    remedy_retry_params=dict(tries=3, delay=0.4, max_delay=4.0,
                             jitter=0.15, backoff=1.8, graceful=False),
)
class MultiDocQAgent(Expression):
    """
    High-level behaviour:
      1. `pre`  – sanity-check query & docs
      2. `act`  – *retrieve* relevant sentences, mutate state
      3. LLM    – generate AnswerWithEvidence (handled by SymbolicAI engine)
      4. `post` – ensure answer & evidence meet semantic rules
      5. `forward`
         • if contract succeeded → return validated LLM object
         • else                   → graceful fallback answer
    """

    # ───────────────────────── init ───────────────────────── #
    def __init__(self, min_coverage: float = 0.55, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.min_coverage = min_coverage          # threshold for `post`
        self.interaction_log: list[dict] = []     # keeps a history of queries

    # ───────────────────────── prompt ─────────────────────── #
    @property
    def prompt(self) -> str:
        """
        A *static* description of what the LLM must do.
        Braces {{like_this}} will be replaced with fields from
        the object produced by `_validate_input`/`_act`.
        """
        return (
            "You are an expert research assistant.\n"
            "Given a QUESTION and a set of RELEVANT_SENTENCES, write a concise "
            "answer.  Cite every passage you use exactly as `(Doc <ID>)`.  "
            "Respond with a JSON object that fits the AnswerWithEvidence schema."
        )

    # ───────────────────────── pre ────────────────────────── #
    def pre(self, input: MultiDocQAInput) -> bool:
        """
        Guard-clauses before we even *touch* the LLM.
        Raise ValueError with human-readable messages – they become corrective
        prompts if `pre_remedy=True`.
        """
        if not input.query.strip():
            raise ValueError("The query must not be empty.")
        if not input.documents:
            raise ValueError("You must supply at least one document.")
        return True  # all good

    # ───────────────────────── act ────────────────────────── #
    def act(self, input: MultiDocQAInput, **kwargs) -> IntermediateRetrieved:
        """
        Lightweight pseudo-retrieval.

        Steps:
          • score each doc by term overlap with the query
          • keep top-k (k ≤ 3)
          • within each, take two most overlapping sentences
          • log the interaction for later analytics
        """
        k = min(3, len(input.documents))
        query_terms = {t.lower() for t in input.query.split()}

        # Score documents by *crude* term overlap
        scored_docs = []
        for doc in input.documents:
            overlap = sum(t in query_terms for t in doc.content.lower().split())
            scored_docs.append((overlap, doc))
        scored_docs.sort(reverse=True, key=lambda x: x[0])

        top_docs = [doc for _, doc in scored_docs[:k]]

        # Extract at most 2 high-overlap sentences from each top doc
        selected_sentences: list[str] = []
        for doc in top_docs:
            sentences = [s.strip() for s in doc.content.split(".") if s.strip()]
            sentences.sort(
                reverse=True,
                key=lambda s: sum(t in query_terms for t in s.lower().split()),
            )
            selected_sentences.extend(sentences[:2])

        # Record what we did (just for analytics / debugging)
        self.interaction_log.append(
            {
                "query": input.query,
                "num_docs": len(input.documents),
                "top_doc_ids": [d.id for d in top_docs],
            }
        )

        # Return a *different* LLMDataModel; this becomes the
        # `current_input` for the output-validation phase.
        return IntermediateRetrieved(
            query=input.query,
            top_docs=top_docs,
            selected_sentences=selected_sentences,
            target_snippet_count=input.max_snippets or 3,
        )

    # ───────────────────────── post ───────────────────────── #
    def post(self, output: AnswerWithEvidence) -> bool:
        """
        Semantic guarantees:
          • non-empty answer
          • coverage ≥ threshold
          • high-coverage → must actually cite evidence
          • evidence list ≤ `target_snippet_count` learned in `act`
        Any violation ⇒ raise ValueError (triggers post-remedy or failure).
        """
        if not output.answer.strip():
            raise ValueError("Answer text is empty.")

        # coverage gate
        if output.coverage_score < self.min_coverage:
            raise ValueError(
                f"Coverage score {output.coverage_score:.2f} "
                f"is below the minimum {self.min_coverage:.2f}."
            )

        # If it claims high coverage but provides no evidence, that's fishy
        if output.coverage_score >= 0.8 and not output.evidence:
            raise ValueError(
                "High coverage claims require at least one evidence snippet."
            )

        # Enforce caller’s snippet bound (act stored it on self._current_input)
        max_allowed = getattr(self, "_current_input", None)
        if (
            isinstance(max_allowed, IntermediateRetrieved)
            and output.evidence
            and len(output.evidence) > max_allowed.target_snippet_count
        ):
            raise ValueError(
                f"Too many snippets ({len(output.evidence)}), "
                f"maximum allowed is {max_allowed.target_snippet_count}."
            )

        return True  # all checks passed

    # ───────────────────────── forward ────────────────────── #
    def forward(self, input: MultiDocQAInput, **kwargs) -> AnswerWithEvidence:
        """
        ALWAYS executed (even if contract failed).

        Success path  → return the LLM-validated object (`self.contract_result`)
        Failure path  → build a polite fallback answer that still matches schema
        """
        # ── happy path ─────────────────────────────────────── #
        if self.contract_successful and self.contract_result:
            return self.contract_result

        # ── fallback (contract failed) ─────────────────────── #
        first_doc = input.documents[0]
        first_sentence = first_doc.content.split(".")[0][:300]  # keep it short
        return AnswerWithEvidence(
            answer=(
                "I’m not confident enough to answer precisely. "
                "Please re-phrase the question or provide more documents."
            ),
            evidence=[DocSnippet(doc_id=first_doc.id, snippet=first_sentence)],
            coverage_score=0.0,
        )

In [3]:
# ▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
#  3.  Mini-demo (only executed when you run the file directly)
# ▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
# ── tiny “corpus” ─────────────────────────────────────── #
docs = [
    Document(
        id="A1",
        content=(
            "Symbolic AI combines formal logic with modern machine learning. "
            "It allows transparent reasoning and explicit knowledge "
            "representation while still benefiting from statistical models."
        ),
    ),
    Document(
        id="B2",
        content=(
            "Vector databases store embeddings of documents. They let users "
            "quickly retrieve text that is semantically similar to a query "
            "vector, enabling high-quality semantic search."
        ),
    ),
    Document(
        id="C3",
        content=(
            "Hybrid search merges sparse keyword techniques and dense vector "
            "similarity, improving recall and precision, especially for "
            "domain-specific collections."
        ),
    ),
]

# ── create agent instance ─────────────────────────────── #
agent = MultiDocQAgent(min_coverage=0.6)

# ── ask a question ────────────────────────────────────── #
question = "Why are vector databases useful for semantic search?"
result = agent(
    input=MultiDocQAInput(
        query=question,
        documents=docs,
        max_snippets=2,  # caller sets stricter evidence limit
    )
)

# ── result ───────────────────────––––––––––––––––––––––– #
print("\nAnswer:\n", result.answer)
print("\nCoverage:", result.coverage_score)
print("\nEvidence:")
for ev in result.evidence:
    print(f" • (Doc {ev.doc_id}) {ev.snippet}")

agent.contract_perf_stats();

2025-06-26 18:35:47.031 | INFO     | symai.strategy:__init__:531 - Initializing contract...
2025-06-26 18:35:47.031 | INFO     | symai.strategy:__init__:541 - Contract initialization complete!
2025-06-26 18:35:47.032 | INFO     | symai.strategy:wrapped_forward:546 - Starting contract execution...
2025-06-26 18:35:47.032 | INFO     | symai.strategy:_validate_input:413 - Starting input validation...
2025-06-26 18:35:47.032 | INFO     | symai.strategy:_validate_input:415 - Validating pre-conditions with remedy...
2025-06-26 18:35:47.032 | SUCCESS  | symai.strategy:_validate_input:423 - Pre-condition validation successful!
2025-06-26 18:35:47.033 | INFO     | symai.strategy:_act:502 - Executing 'act' method on MultiDocQAgent…
2025-06-26 18:35:47.033 | SUCCESS  | symai.strategy:_act:523 - 'act' method executed successfully!
2025-06-26 18:35:47.033 | INFO     | symai.strategy:_validate_output:448 - Starting output validation...
2025-06-26 18:35:47.033 | INFO     | symai.strategy:_validate_ou

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  You are an expert research assistant.                                                                          │
│  Given a QUESTION and a set of RELEVANT_SENTENCES, write a concise answer.  Cite every passage you use exactly  │
│  as `(Doc <ID>)`.  Respond with a JSON object that fits the AnswerWithEvidence schema.                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Input data model ───────────────────────────────────────────╮
│                                                                                                       │
│  [[Schema]]                                                                                           │
│  - "query" (string, required): The original question from the user.                                   │
│  - "top_docs" (array of nested object (Document), required): Top-k most relevant documents.           │
│    - Nested fields:                                                                                   │
│    - "id" (string, required): Unique identifier of the document.                                      │
│    - "content" (string, required): The full raw text of the document.                                 │
│  - "selected_sentences" (array of string, required): Sentences deemed most relevant to the query.     │
│  - "target_snippet_count" (integer, required): Upper bound on evidence snippets (copied from input).  │
│                                                                                                       │
│  [[Definitions]]                                                                                      │
│  - Document:                                                                                          │
│    - "id" (string, required): Unique identifier of the document.                                      │
│    - "content" (string, required): The full raw text of the document.                                 │
│                                                                                                       │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Output data model ───────────────────────────────────────────────╮
│                                                                                                                 │
│  [[Schema]]                                                                                                     │
│  - "answer" (string, required): Concise, stand-alone answer.                                                    │
│  - "evidence" (array of nested object (DocSnippet), required): Cited supporting passages.                       │
│    - Nested fields:                                                                                             │
│    - "doc_id" (string, required): ID of the document the snippet comes from.                                    │
│    - "snippet" (string, required): A short excerpt supporting the answer.                                       │
│  - "coverage_score" (number, required): LLM-estimated fraction of answer that is directly supported by the      │
│  evidence (0 = no support, 1 = fully supported).                                                                │
│                                                                                                                 │
│  [[Definitions]]                                                                                                │
│  - DocSnippet:                                                                                                  │
│    - "doc_id" (string, required): ID of the document the snippet comes from.                                    │
│    - "snippet" (string, required): A short excerpt supporting the answer.                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-06-26 18:35:50.420 | INFO     | symai.strategy:forward:305 - Prepared 3 remedy seeds for validation attempts…
2025-06-26 18:35:50.422 | INFO     | symai.strategy:forward:310 - Attempt 1/3: Attempting validation…
2025-06-26 18:35:50.422 | SUCCESS  | symai.strategy:forward:360 - Validation completed successfully!
2025-06-26 18:35:50.425 | SUCCESS  | symai.strategy:_validate_output:461 - Type successfully created!
2025-06-26 18:35:50.426 | INFO     | symai.strategy:_validate_output:464 - Validating post-conditions with remedy...
2025-06-26 18:35:50.427 | SUCCESS  | symai.strategy:_validate_output:472 - Post-condition validation successful!
2025-06-26 18:35:50.427 | INFO     | symai.strategy:wrapped_forward:596 - Executing original forward method...
2025-06-26 18:35:50.429 | SUCCESS  | symai.strategy:wrapped_forward:624 - Contract validation successful!



Answer:
 Vector databases are useful for semantic search because they store embeddings of documents and allow for the quick retrieval of text that is semantically similar to a query vector, which enables high-quality semantic search (Doc B2).

Coverage: 1.0

Evidence:
 • (Doc B2) Vector databases store embeddings of documents.
 • (Doc B2) They let users quickly retrieve text that is semantically similar to a query vector, enabling high-quality semantic search.


                              Contract Execution Summary (1 Forward Calls)                              
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Operation         ┃ Count ┃ Total Time (s) ┃ Mean (s) ┃ Std Dev (s) ┃ Min (s) ┃ Max (s) ┃ % of Total ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━┩
│ Input Validation  │     1 │          0.000 │    0.000 │       0.000 │   0.000 │   0.000 │       0.0% │
│ Act Execution     │     1 │          0.000 │    0.000 │       0.000 │   0.000 │   0.000 │       0.0% │
│ Output Validation │     1 │          3.392 │    3.392 │       0.000 │   3.392 │   3.392 │      99.9% │
│ Forward Execution │     1 │          0.000 │    0.000 │       0.000 │   0.000 │   0.000 │       0.0% │
│ Overhead          │     1 │          0.005 │    0.005 │       0.000 │   0.000 │   0.000 │       0.1% │
│ Total Execution   │   N/A │          3.397 │    3.397 │       0.000 │   3.397 │   3.397 │     100.0% │
└───────────────────┴───────┴────────────────┴──────────┴─────────────┴─────────┴─────────┴────────────┘